In [104]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt

In [105]:
%matplotlib inline
import pandas as pd
import scipy.stats as stats

In [106]:
df = pd.read_csv("CM_Dummy_Data/cm_dummy.csv")
df=df.fillna(0)
df.head()

,Team Code,Team Name,Conference,Game Code,Date,Rush Yard,Pass Yard,Total Yards,Bye,Off Points,Def Points,Result
0,8,Alabama,Southeastern Conference,8.070000e+12,8/31/2013,96,110,206,0,21,7,W
1,8,Alabama,Southeastern Conference,8.070000e+12,9/14/2013,234,334,568,0,37,20,W
2,8,Alabama,Southeastern Conference,1.560000e+14,9/21/2013,66,272,338,0,21,3,W
3,8,Alabama,Southeastern Conference,4.330000e+14,9/28/2013,254,180,434,0,32,16,W
4,8,Alabama,Southeastern Conference,2.540000e+14,10/5/2013,181,296,477,0,36,13,W


In [107]:
#following block determines the desired data (yard and points for games after bye week vs. non - bye week games)

#looked up a function online to pull year information from a date. Through that, found out the Date column wasn't a date in pandas. Converted first
#PLZ NOTE , MIGHT WANT TO TAKE THIS OUT 
df["Year"]=df["Date"].apply(pd.to_datetime).dt.year

#determining the average amount of yards and points after a bye week per team
byeWeeks=df[df["Bye"]=="x"]
ByeWeekOffYards = byeWeeks.groupby("Team Name")["Total Yards"].mean()
ByeWeekOffPts = byeWeeks.groupby("Team Name")["Off Points"].mean()
ByeWeekDefPts = byeWeeks.groupby("Team Name")["Def Points"].mean()

#determining the average amount of yards and points after on a non-bye week per team
non_byeWeeks=df[df["Bye"]!="x"]
non_ByeWeekOffYards = non_byeWeeks.groupby("Team Name")["Total Yards"].mean()
non_ByeWeekOffPts = non_byeWeeks.groupby("Team Name")["Off Points"].mean()
non_ByeWeekDefPts = non_byeWeeks.groupby("Team Name")["Def Points"].mean()

#determining the total number of W's after bye weeks
#PLEASE NOTE, IF NOT ALL TEAMS HAVE A BYE WEEK W, WE MIGHT HAVE TO ADD THEM TO THE BOTTOM OF THE ARRAY, NEED TO CHECK
byeWeek_Wins=byeWeeks[byeWeeks["Result"] == "W"]
byeWeek_Wins = byeWeek_Wins.groupby("Team Name")["Result"].count()

In [199]:
df_ByeWeekStats = pd.merge(ByeWeekOffYards,ByeWeekOffPts, on=["Team Name"])
df_ByeWeekStats = pd.merge(df_ByeWeekStats,ByeWeekDefPts, on=["Team Name"])

df_Non_ByeWeekStats = pd.merge(non_ByeWeekOffYards,non_ByeWeekOffPts, on=["Team Name"])
df_Non_ByeWeekStats = pd.merge(df_Non_ByeWeekStats,non_ByeWeekDefPts, on=["Team Name"])

Wins = df.groupby("Team Name")["Team Code"].count()
df_wins = pd.merge(Wins,byeWeek_Wins, on=["Team Name"], how = "left")
df_wins=df_wins.fillna(0)
df_wins = df_wins.drop(["Team Code"], axis=1)

df_stats = pd.merge(df_ByeWeekStats,df_Non_ByeWeekStats, on=["Team Name"])
df_stats = pd.merge(df_stats,df_wins, on=["Team Name"])

df_stats = df_stats.rename(columns={"Total Yards_x": "Bye Week - Average Yards", "Off Points_x": "Bye Week - Average Offensive Points", \
                                   "Def Points_x": "Bye Week - Average Defensive Points", "Total Yards_y": "Non Bye Week - Average Yards", \
                                    "Off Points_y": "Non Bye Week - Average Offensive Points", "Def Points_y": "Non Bye Week - Average Defensive Points",\
                                   "Result" : "Wins after Bye Week"})